In [141]:
import time
import pandas as pd
import numpy as np
import re
import Levenshtein
import ast
import matplotlib.pyplot as plt
import yaml

from huggingface_hub import HfApi
from huggingface_hub import ModelSearchArguments, DatasetSearchArguments, ModelCard
from huggingface_hub import hf_hub_url, get_hf_file_metadata
from difflib import SequenceMatcher

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import matplotlib

pd.options.mode.chained_assignment = None

In [142]:
def read_df_processed():
    df = pd.read_csv('datasets/HFTotalProcessed.csv')
    df = df.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1)
    df['library_name'] = df['library_name'].apply(lambda libraries:  ast.literal_eval(libraries) if not isinstance(libraries, list) else libraries)
    df['datasets'] = df['datasets'].apply(lambda datasets: [''] if pd.isnull(datasets) else [datasets] if '[' not in datasets else ast.literal_eval(datasets))

    return df

df = read_df_processed()

def read_df_clean():
    df = pd.read_csv('datasets/HFClean.csv')
    df = df.drop(['Unnamed: 0'], axis=1)
    df['library_name'] = df['library_name'].apply(lambda libraries:  ast.literal_eval(libraries) if not isinstance(libraries, list) else libraries)
    df['datasets'] = df['datasets'].apply(lambda datasets: [''] if pd.isnull(datasets) else [datasets] if '[' not in datasets else ast.literal_eval(datasets))

    return df

df_clean = read_df_clean()

/tmp/ipykernel_39114/175888158.py:2: DtypeWarning: Columns (6,7,8,9,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('datasets/HFTotalProcessed.csv')


In [143]:
# wanted_columns = ['modelId', 'datasets_size', 'co2_eq_emissions', 'source', 'training_type', 'geographical_location',
#                   'downloads', 'likes', 'library_name', 'created_at', 'co2_reported', 'domain',
#                   'year_month', 'size', 'size_efficency', 'performance_score']

wanted_columns = [col for col in df.columns if not col.startswith('is_')]

df = df[wanted_columns]

In [144]:
import pandas as pd

# Load your data
df1 = df
df2 = pd.read_csv('datasets/HFClean.csv')

# Merge the dataframes
merged = df1.merge(df2, on='modelId', how='left', suffixes=('', '_y'))

# Replace _x columns (from HFStreamlit.csv) with _y columns (from HFClean.csv) when _y is not null
for column in merged.columns:
    if '_y' in column:
        merged[column.replace('_y', '')] = merged[column].where(merged[column].notnull(), merged[column.replace('_y', '')])

# Drop _y columns
df = merged[df1.columns]




In [145]:
df['domain'] = df['domain'].fillna('Not Specified')
df['training_type'] = df['training_type'].fillna('Not Specified')
df['source'] = df['source'].fillna('Not Specified')
df['geographical_location'] = df['geographical_location'].fillna('Not Specified')
df['hardware_used'] = df['hardware_used'].fillna('Not Specified')

In [146]:
def combine_sources(source, auto):

    if auto:
        return 'AutoTrain'
    if source == 'code carbon':
        return 'Code Carbon'
    if 'mlco2' in source or 'ML CO2' in source:
        return 'MLCO2'
    if 'BLOOM' in source:
        return 'Article'
    if 'Google Cloud' in source:
        return 'Google Cloud Footprint'
    
    return 'Not Specified'

def combine_location(location):

    if 'East US' in location:
        return 'East US'
    if location == 'Frankfurt an Main, Germany (500-600 gCO2eq/kWh)':
        return 'Frankfurt an Main, Germany'
    return location



def combine_training_type(training_type):

    if 'fine' in training_type:
        return 'fine-tuning'
    if 'pre' in training_type:
        return 'pretraining'
    
    return 'Not Specified'


In [147]:
df['source'] = df.apply(lambda row: combine_sources(row['source'], row['auto']), axis=1)
df['geographical_location'] = df['geographical_location'].apply(lambda location: combine_location(location))
df['training_type'] = df['training_type'].apply(lambda training_type: combine_training_type(training_type))
df['size_efficency'] = df['size'] / df['co2_eq_emissions']
df['datasets_size_efficency'] = df['datasets_size'] / df['co2_eq_emissions']
df['downloads'] = df['downloads'].astype(int)
df['likes'] = df['likes'].astype(int)
df['co2_reported'] = df['co2_reported'].astype(int)

df['created_at'] = pd.to_datetime(df['created_at'])

df = df[df['co2_reported'] == True]


# # format the date part
# df['created_at'] = df['created_at'].apply(lambda x: f"{x.month}/{x.day}/{x.year}")

# df['created_at'] = pd.to_datetime(df['created_at'])

# # get the date part
df['created_at'] = df['created_at'].dt.date

def create_performance_metrics(row):
    return {'accuracy': row['accuracy'], 'f1': row['f1'], 'rouge1': row['rouge1'], 'rouge2': row['rougeL']}

df['performance_metrics'] = df.apply(create_performance_metrics, axis=1)

In [148]:
df = df.rename(columns={'hardware_used': 'environment'})

Let's delete unnecessary attributes

In [149]:
wanted_columns = ['modelId', 'datasets', 'datasets_size', 'co2_eq_emissions', 'co2_reported', 'source', 'training_type', 'geographical_location', 'environment', 'performance_metrics', 'performance_score',
                  'downloads', 'likes', 'library_name', 'domain', 'size', 'created_at', 'size_efficency', 'datasets_size_efficency']


df = df[wanted_columns]

In [150]:
df.to_csv('HFStreamlit.csv', index=False)

In [52]:
from datetime import datetime
import pytz

# get current datetime
now = datetime.now(pytz.utc)

# format datetime
formatted_now = now.strftime('%Y-%m-%d %H:%M:%S%z')
print(formatted_now)

2023-06-11 03:38:41+0000


In [38]:

def read_df_processed():
    df = pd.read_csv('datasets/HFStreamlit.csv')
    df['library_name'] = df['library_name'].apply(lambda libraries:  ast.literal_eval(libraries) if not isinstance(libraries, list) else libraries)
    # df['performance_metrics'] = df['performance_metrics'].apply(lambda metrics_dict:  ast.literal_eval(metrics_dict) if isinstance(metrics_dict, str) else metrics_dict)

    return df


df = read_df_processed()